# Tutoriel sur pyspark et MapReduce

## 1. Configuration de l'environnement Spark

In [1]:
# S'assurer d'avoir correctement installé pyspark et Spark
# Importer quelques classes Spark pour initialiser notre environnement
from pyspark import SparkContext, SparkConf
 
# Initialiser Spark
conf = SparkConf()
sc = SparkContext(conf=conf)

## 2. Création d'un RDD de données numériques

In [2]:
# Créer un jeu de données
data = list(range(10)) ## [0, 1, 2, 3, 4, 5, 6, 7, 9]
# Créer un RDD
rdd = sc.parallelize(data)
 
print(type(rdd), '\t', rdd.collect())

<class 'pyspark.rdd.RDD'> 	 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


## 3. Création d'un RDD de chaînes de caractères

In [3]:
# Créer un jeu de données
data = ['Ledatascientist', 'le', 'top', 'des', 'blogs', 'français', 'en', 'Data', 'Science']
# Créer un RDD
rdd = sc.parallelize(data, numSlices=10).glom()
 
print(type(rdd), '\t', rdd.collect())

<class 'pyspark.rdd.PipelinedRDD'> 	 [[], ['Ledatascientist'], ['le'], ['top'], ['des'], ['blogs'], ['français'], ['en'], ['Data'], ['Science']]


## 4. Application
Nous voulons **calculer le nombre d’occurrences des mots dans un texte quelconque**.

### 4.1. Lecture de notre fichier TXT
Nous créerons un RDD à partir d’un fichier ```example.txt``` qui contient notre article sur [les bases du Machine Learning](https://ledatascientist.com/machine-learning-les-bases/).

In [4]:
# Créer un RDD à partir d'un fichier TXT
rdd = sc.textFile("./example.txt")
# Affichage du premier élément de notre RDD
print(type(rdd), '\n', rdd.first())

<class 'pyspark.rdd.RDD'> 
 Machine Learning ou apprentissage automatique est l’un des champs de l’intelligence artificielle (IA). Ce domaine apparu tout au début des années 1960 va devenir de nos jours très incontournable au XXIe siècle pour la résolution de problèmes concrets et complexes.


### 4.2. Définition de notre fonction de *mapping*

In [5]:
def process(sentence):
    """
    Supprime les ponctuations d'une phrase, la splitte en liste 
    et retourne la liste des mots (en miniscule) dont la longueur est supérieure à 3.
     
    Parameter
    ----------
    sentence: str
        Une phrase.
     
    Returns
    --------
    words: list
        La liste des mots de la phrase "sentence" dont la longueur est supérieure à 3.
    """
     
    ## Suppression des ponctuations
    for p in "`!()-_[]{};:’,./?\"":
        sentence = sentence.replace(p, " ")
     
    ## Split de la phrase
    list_ = sentence.split()
     
    ## Liste des mots de longueur supérieure à 3
    words = [word.lower() for word in list_ if len(word) > 3]
     
    ## Sortie de notre fonction
    return words

### 4.3. Création de notre RDD transformé

In [6]:
# Créer le nouveau RDD
rdd_processed = rdd.flatMap(process)
# Affichage des quinze premiers éléments de notre RDD
rdd_processed.take(15) ## take(1) == first()

['machine',
 'learning',
 'apprentissage',
 'automatique',
 'champs',
 'intelligence',
 'artificielle',
 'domaine',
 'apparu',
 'tout',
 'début',
 'années',
 '1960',
 'devenir',
 'jours']

### 4.4. Application du MapReduce
Nous affichons la liste des **cinq mots les plus récurrents du texte**.

In [7]:
# Map - Reduce - Sort
rdd_final = rdd_processed.map(lambda word: (word, 1))\
                            .reduceByKey(lambda a, b: a+b)\
                                .sortBy(lambda x: -x[1])
 
# Afficher les cinq premiers éléments
rdd_final.take(5)

[('machine', 17),
 ('apprentissage', 13),
 ('learning', 12),
 ('supervisé', 10),
 ('régression', 9)]